In [1]:
import os, sys, json
import itertools, importlib
import asyncpg

from edcompanion.eddbreader import edc_dbfilereader, edc_dbfile_process

In [ ]:
pgsql_params = dict(
    dsn=os.getenv("PGSQL_URL"),
    server_settings={'search_path': "eddb"}
)
pgpool = await asyncpg.create_pool(**pgsql_params)

In [2]:
dbname_full = 'systems.json.gz'
dbname_day = 'systems_1day.json.gz'
dbname_week = 'systems_1week.json.gz'
dbname_month = 'systems_1month.json.gz'

In [3]:
dataset = []
for item in edc_dbfilereader(os.path.join('data', dbname_day)):
    dataset.append(item)

Reading data/systems_1day.json.gz, 2.5 Mb in approx 1 chunks
1/1	100.00%, 261580 /s,     76758 systems,   0.0 seconds remaining
Empty chunk -> Done! Imported 76758 systems in 0.3 seconds

0.295181999 seconds 76758 systems, per system 3.85 us

In [4]:
dataset[0]

{'id64': 2326687,
 'name': 'HD 192281',
 'mainStar': 'O (Blue-White) Star',
 'coords': {'x': -4023.53125, 'y': 230.875, 'z': 896.46875},
 'updateTime': '2024-04-13 21:57:30+00'}

In [ ]:
await pgpool.execute(f"DROP TABLE eddb.systems;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS eddb.systems (
        id64 BIGINT NOT NULL,
        x DOUBLE PRECISION  NOT NULL,
        y DOUBLE PRECISION  NOT NULL,
        z DOUBLE PRECISION  NOT NULL,
        name TEXT NOT NULL,
        class TEXT NOT NULL
    );
""")
